# Pipeline step by step Implementation

In [1]:
import os
from dataclasses import dataclass
from autogen import ConversableAgent
from dotenv import load_dotenv

## Define the System Messages

In [2]:
def get_query_enrichment_system_message() -> str:
    return """You are a car expert who helps gather detailed information about DIY car repair queries.
    
    Your primary responsibilities:
    1. Identify missing information about:
       - Car brand and model
       - Year of manufacture
       - Specific part or system involved
       - Nature of the problem or maintenance task
    
    Conversation guidelines:
    - Ask ONE question at a time
    - Focus on the most critical missing information first
    - Prioritize questions in this order:
        1. Vehicle identification (make/model/year)
        2. Specific part or system
        3. Problem details
    
    Question format:
    - Keep questions concise and specific
    - Use automotive terminology appropriately
    - Provide options when applicable (e.g., "Is this a sedan or SUV?")
    
    Example questions:
    - "What make and model is your car?"
    - "What year was your vehicle manufactured?"
    - "Which specific part are you working with?"
    - "Can you describe the symptoms you're experiencing?"
    
    Always maintain a professional and helpful tone while gathering information. Once all information is gathered, provide a summary in this format:
    
    SUMMARY:
    Vehicle: [Year] [Brand] [Model]
    Part/System: [Specific part]
    Task: [Detailed description of the problem/task]
    Additional Notes: [Any relevant details provided by user]
    """

In [3]:
def get_response_enhancement_system_message() -> str:
    return """You are an expert at explaining technical car terminology and making DIY instructions accessible.
    
    Your primary responsibilities:
    1. Identify technical terms and jargon in responses
    2. Add clear explanations for complex concepts
    3. Format responses for maximum readability
    4. Maintain technical accuracy while improving clarity

    Enhancement guidelines:
    1. Technical Term Identification:
       - Identify specialized automotive terms
       - Flag industry-specific abbreviations
       - Note complex procedures
    
    2. Explanation Format:
       - Add a "Technical Terms Explained" section at the end
       - Use simple, everyday language in explanations
       - Include relevant analogies when helpful
    
    3. Formatting:
       - Break down complex procedures into steps
       - Use bullet points for lists
       - Highlight important safety warnings
    
    4. Maintain Original Content:
       - Keep original technical information intact
       - Add explanations without removing details
       - Preserve specific part numbers and specifications
    
    Example format:
    [Original technical response]
    
    Technical Terms Explained:
    - Term 1: Simple explanation
    - Term 2: Simple explanation with analogy
    
    Safety Notes:
    [Any relevant safety information]."""

In [4]:
def get_rag_agent_system_message() -> str:
    return """You are an expert capable at searching the knowledge base for repair instructions and safety information.
    
    Your task is to:
    1. Take enriched car repair queries
    2. Use the query_car_knowledge function to search the knowledge base
    3. Return relevant repair instructions and safety information
    
    Always use the query_car_knowledge function to get information.
    Never make up information or rely on general knowledge.
    """

In [5]:
def get_entry_agent_system_message() -> str:
    return """You are the coordinator for a Car DIY help system. Your role is to:
    1. Receive and route user queries through the system
    2. Manage the flow between query enrichment, RAG lookup, and response enhancement
    3. Ensure all necessary information is collected before proceeding to the next step
    4. Maintain context throughout the conversation
    
    Guidelines:
    - Always verify that enriched queries contain all necessary vehicle information
    - Pass complete context to the RAG system
    - Ensure enhanced responses maintain original technical accuracy
    - Keep track of the conversation state and manage transitions between agents
    """

## Define Prompts

In [6]:
def get_query_enrichment_prompt(query: str) -> str:
    prompt = (
        f"The user has requested help to DIY a car repair. The inquired query summary can be found here: '{query}'. "
        f"Please suggest a function call to collect more information for this query, using the function 'ask_question'."
        f"Once the function call has been made, please format the response to include a summary of user's need."
        )
    return prompt

In [7]:
def get_rag_agent_prompt(query: str) -> str:
    prompt = (
        f"The user has requested help to DIY a car repair. The inquired query summary can be found here: '{query}'. "
        f"Please suggest a function call to fetch the information for this query, using the function 'query_knowledge_base'."
        f"Once the function call has been made, please Format the response to include:"
        f"  1. Step-by-step instructions"
        f"  2. Required tools and parts"
        f"  3. Safety precautions"
        f"  4. Common pitfalls to avoid"
        )
    return prompt

# Set up RAG

In [8]:
from agents.agentic_rag_2 import MultiDocumentRAG, root_dir # Import your RAG system
    
# Initialize RAG system
UPLOAD_DIR = os.path.join(root_dir, "backend/uploads")
PERSIST_DIR = os.path.join(root_dir, "backend/storage")

rag = MultiDocumentRAG(UPLOAD_DIR, PERSIST_DIR)

rag.setup_indicies()

files = os.listdir(UPLOAD_DIR)
rag.create_indices(files)
rag.setup_agent()

Loaded index for Tesla_Model_3_Brakes.txt
Loaded index for Tesla_Model_3_Seats.txt
No new files to create indices for
Name: Seat_Cushion_Lower_2nd_Row_Removal_and_Replacement_Guide
Description: The document provides detailed instructions for removing and replacing various components in a Tesla Model 3, including the 2nd-row lower seat cushion, brake components, driver footwell duct, knee airbag cover, footwell cover, 12V power connection, and rear underhood apron.
Name: Front_Passenger_Seat_Back_Rest_Replacement_Procedure
Description: This document provides detailed instructions for removing and replacing the front passenger seat back rest in a Tesla Model 3. It includes steps for removing various components such as the lumbar support, recliner motor, and headrest post guides, as well as precautions to ensure safety during the procedure. The document also outlines the installation process, including torque specifications for bolts and recommended tools for the job.


## Set up function calls

### knowledge base query

the function call will be determined by rag_agent using the summary from entry point

In [9]:
def query_knowledge_base(query: str) -> str:
    """Function to query the RAG system"""
    try:
        response = rag.query(query).response
        return response
    except Exception as e:
        return f"Error querying knowledge base: {str(e)}"

### post query enhancement
`TODO` - the function call will replace the effort of prompt engineering once we have the query using `response_enhancement_agent`

## Set up Group Chat

In [10]:
from autogen import ConversableAgent, GroupChat, GroupChatManager

def setup_groupchat():
    load_dotenv()
    llm_config = {"config_list": [{"model": 'gpt-4', "api_key": os.environ["OPENAI_API_KEY"]}]}
    
    # Human proxy - for user interaction
    human_proxy = ConversableAgent(
        "human_proxy",
        llm_config=False,
        human_input_mode="ALWAYS"
    )
    
    # Query enrichment agent - focuses on gathering information
    query_enrich_agent = ConversableAgent(
        "query_enrich_agent",
        system_message="""You are a car expert who helps gather detailed information about DIY car repair queries.
        
        Your task is to have a conversation with human_proxy to gather:
        1. Car brand and model
        2. Year of manufacture
        3. Specific part or system involved
        4. Nature of the problem or maintenance task

        Guidelines:
        - Ask ONE question at a time directly to human_proxy
        - If any information is already provided in the query, don't ask for it again
        - Wait for human_proxy's response before asking the next question
        - Once all information is gathered, provide a summary in this format:
        
        SUMMARY:
        Vehicle: [Year] [Brand] [Model]
        Part/System: [Specific part]
        Task: [Detailed description of the problem/task]
        Additional Notes: [Any relevant details provided by user]

        After providing the SUMMARY, stop and let rag_agent take over.""",
        llm_config=llm_config
    )
    
    # RAG agent - handles knowledge base queries
    rag_agent = ConversableAgent(
        "rag_agent",
        system_message="""You handle knowledge base queries for car repair information.
        When you see a SUMMARY from query_enrich_agent:
        1. Extract the relevant information from the SUMMARY
        2. Use the query_knowledge_base function to get repair instructions
        3. Share the retrieved instructions with response_enhancement_agent""",
        llm_config=llm_config
    )
    
    # Response enhancement agent - explains technical terms
    response_enhancement_agent = ConversableAgent(
        "response_enhancement_agent",
        system_message="""You enhance technical repair instructions with explanations.
        When you receive instructions from rag_agent:
        1. Identify all technical terms
        2. Add clear explanations while keeping original instructions intact
        3. Format your response as:
        
        REPAIR INSTRUCTIONS:
        [Original instructions]

        TECHNICAL TERMS EXPLAINED:
        - [Term 1]: [Simple explanation with everyday examples]
        - [Term 2]: [Simple explanation with everyday examples]""",
        llm_config=llm_config
    )
    
    # Only register the query_knowledge_base function with RAG agent
    rag_agent.register_for_llm(
        description="Queries the car repair knowledge base using the enriched query"
    )(query_knowledge_base)
    rag_agent.register_for_execution()(query_knowledge_base)

    # Create GroupChat with clear flow
    groupchat = GroupChat(
        agents=[human_proxy, query_enrich_agent, rag_agent, response_enhancement_agent],
        messages=[],
        max_round=20  # Increased to allow for multiple questions from query_enrich_agent
    )

    # Manager focuses purely on orchestration
    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        system_message="""You are managing a car repair help conversation. Follow this EXACT sequence:
        
        CONVERSATION FLOW:
        1. START: Let query_enrich_agent begin with a question to human_proxy
        2. INFORMATION GATHERING:
        - Let query_enrich_agent and human_proxy interact
        - query_enrich_agent should ask ONE question at a time
        - Continue until query_enrich_agent provides a SUMMARY
        3. RAG QUERY:
        - When a SUMMARY appears, direct rag_agent to process it
        - Ensure rag_agent uses query_knowledge_base function
        4. ENHANCEMENT:
        - When rag_agent provides instructions, direct response_enhancement_agent
        - Wait for response with both REPAIR INSTRUCTIONS and TECHNICAL TERMS EXPLAINED
        
        SPEAKER CONTROL:
        - query_enrich_agent starts with a friendly greeting and first question
        - Manage turn-taking between agents based on their responses
        - Ensure each agent completes their task before moving to the next phase
        
        TERMINATION CONDITIONS:
        - End when response includes both REPAIR INSTRUCTIONS and TECHNICAL TERMS EXPLAINED
        - End if maximum rounds (20) are reached
        
        Your role is to direct traffic between agents and ensure the conversation follows this sequence."""
    )

    return manager, groupchat


def run_car_diy_chat():
    manager, groupchat = setup_groupchat()
    # Start the chat
    manager.initiate_chat(
        manager,
        message="Hello! I am here to help you with your DIY needs"
    )

    # Return the final enhanced response
    chat_history = groupchat.messages
    return chat_history

In [15]:
chat_history = run_car_diy_chat()

chat_manager (to chat_manager):

Hello! I am here to help you with your DIY needs

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to chat_manager):

I want to remove my driver seat

--------------------------------------------------------------------------------

Next speaker: query_enrich_agent


>>>>>>>> USING AUTO REPLY...
query_enrich_agent (to chat_manager):

Sure, I can guide through it. Could you please tell me which brand and model your car is?

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to chat_manager):

tesla model 3

--------------------------------------------------------------------------------

Next speaker: query_enrich_agent


>>>>>>>> USING AUTO REPLY...
query_enrich_agent (to chat_manager):

Great! Could you tell me the manufacturing year of your Tesla Model 3?

------------------------------------------------------

In [13]:
for message in chat_history:
    print(f"\n{message['name']}: \n\t{message['content']}")


chat_manager: 
	Hello! I am here to help you with your DIY needs

human_proxy: 
	I want to replace my front seat of my car

query_enrich_agent: 
	Absolutely, I'd be glad to help! Can you please tell me which brand and model of car you're referring to?

human_proxy: 
	Tesla model 3 

query_enrich_agent: 
	That's great! When was your Tesla Model 3 manufactured?

human_proxy: 
	2023

query_enrich_agent: 
	Thank you for the information. Now, could you please explain what exactly the problem is with the current front seat?

human_proxy: 
	no reason, I just want to replace it

query_enrich_agent: 
	SUMMARY:
Vehicle: 2023 Tesla Model 3
Part/System: Front Seat
Task: Replacement of the front seat without any existing problem.
Additional Notes: The customer wants to replace the seat without any specific issue. 

Now, I am passing it on to the agent who will guide you on how to replace your front seat.

rag_agent: 
	None

rag_agent: 
	To replace the front seat of a 2023 Tesla Model 3, you will n

In [14]:
print(len(chat_history))

12


In [40]:
rag.shutdown()